In [1]:
import os
import vtk
import numpy as np
from utilities import ReadVTPFile, WriteVTPFile
from vtk.util.numpy_support import vtk_to_numpy, numpy_to_vtk

In [20]:
InputFolder = "/Users/ana/Documents/AnahitaSeresti/05_PrePostCABG/RayCasting/SU04A/PolarMap"
polar_map = os.path.join(InputFolder, "MyocardiumPolarMap.vtp")
PolarMap = ReadVTPFile(polar_map)

In [21]:
from sklearn.cluster import KMeans

MBFPolarValue = vtk_to_numpy(PolarMap.GetPointData().GetArray("MBFProfile"))
#MBFPolarValue = MBFPolarValue[(MBFPolarValue < 1000) & (MBFPolarValue > -100)]
X = MBFPolarValue.reshape(-1,1)
kmeans = KMeans(n_clusters=2, random_state=5)
kmeans.fit(X)
labels = kmeans.labels_

/usr/local/anaconda3/envs/vtkfenics/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [22]:
len(labels[labels==1])

220326

In [23]:
means = [MBFPolarValue[labels == i].mean() for i in range(2)]
mean_healthy = np.max(means)
mean_ischemic = np.min(means)
means

[19.008059, 40.125057]

In [24]:
ischemic_label = np.argmin(means)
ischemic_label

0

In [25]:
ischemia_mask = labels == ischemic_label
ischemia_mask

array([ True,  True,  True, ..., False,  True, False])

In [26]:
ischemia_array = numpy_to_vtk(ischemia_mask.astype(np.uint8), deep=True)
ischemia_array.SetName("IsIschemic")
PolarMap.GetPointData().AddArray(ischemia_array)

ofile = os.path.join(InputFolder, "MyocardiumPolarMapIschemic.vtp")
WriteVTPFile(ofile, PolarMap)

In [27]:
InputFolder = "/Users/ana/Documents/AnahitaSeresti/05_PrePostCABG/RayCasting/SU04B/PolarMap"
polar_map = os.path.join(InputFolder, "MyocardiumPolarMap.vtp")
PolarMap = ReadVTPFile(polar_map)

In [28]:
MBFPolarValue = vtk_to_numpy(PolarMap.GetPointData().GetArray("MBFProfile"))
d_ischemic = np.abs(MBFPolarValue - mean_ischemic)
d_healthy = np.abs(MBFPolarValue - mean_healthy)

ischemia_mask = (d_ischemic < d_healthy)
ischemia_array = numpy_to_vtk(ischemia_mask.astype(np.uint8), deep=True)
ischemia_array.SetName("IsIschemic")
PolarMap.GetPointData().AddArray(ischemia_array)

ofile = os.path.join(InputFolder, "MyocardiumPolarMapIschemic.vtp")
WriteVTPFile(ofile, PolarMap)